In [2]:
# Import dependencies
from unityagents import UnityEnvironment
import numpy as np
from collections import deque, defaultdict,namedtuple
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [4]:
# Initialise environment
env = UnityEnvironment(file_name='Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [6]:
from ddpg_agent import Agent

# Initialise Agent and define hyperparameters
agent = Agent(state_size = brain.vector_observation_space_size, action_size = brain.vector_action_space_size,
              seed = 0, buffer_size = 10000, batch_size = 256, learning_rate_actor = 1e-3, 
              learning_rate_critic = 1e-3, e = 1e-5, beta = 0.4, a = 0.5)

In [ ]:
%%time
from replay import ExperienceReplay

def ddpg(n_episodes=20000):
    
    scores = []  # Save scores in a list                      
    scores_window = deque(maxlen=100)  # Sliding window list with max length of 100 
                    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations 
        
        score = np.zeros(20)
        # Keeps track of time step
        t = 0
        for t in range(3000):
            # choosing next action
            actions = agent.act(states)
            # running next action through env to get next_state, reward, done
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards 
            dones = env_info.local_done
            # Model weights are updated every X time steps
            agent.step(states, actions, rewards, next_states, dones, t, 20, 0.9)
            # S' -> S
            states = next_states
            score += rewards
            # Adds 1 to time step
            t+=1
            # Break while loop if done
            if np.any(dones):
                break 
        
        scores.append(score.tolist())
        scores_window.append(np.mean(scores[-100:], axis=0))     
         
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=30:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_per.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_per.pth')
            break
    return scores

scores = ddpg()

Episode 19	Average Score: 0.52